In [3]:
%matplotlib inline

from pathlib import Path
from datetime import date

import numba
import rasterio
import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from cartopy import crs
from scipy import stats

import weather_ecmwf
import population_tools

from config import DATA_SRC, WEATHER_SRC, POP_DATA_SRC

In [5]:
DAILY_TOTAL_OUT = WEATHER_SRC / 'ecmwf' / 'daily_total_precipitation'
DAILY_TOTAL_OUT.mkdir(parents=True)

In [7]:
PPT_5DAY_OUT  = WEATHER_SRC / 'ecmwf' / '5day_precipitation'
PPT_5DAY_OUT.mkdir(parents=True)

In [6]:
for f in (WEATHER_SRC / 'ecmwf' / 'daily_ppt').glob('*_daily_precipitation.nc'):
    year = int(f.name.split('_')[0])
    print(year)
    ppt = xr.open_dataset(f,
                          chunks={'latitude': 122, 
                                  'longitude': 240})
    ppt = ppt.resample(time='1D').sum() * 1000 # in meters
    ppt.to_netcdf(DAILY_TOTAL_OUT / f'{year}_daily_precipitation.nc')
    

2000
1990
1998
1996
2015
2012
1984
1989
2001
2007
2011
1993
1980
2008
1991
2005
1985
2014
1987
2010
1988
2002
1982
2004
2013
1994
1999
2006
1981
2017
1983
1986
1992
2016
1997
2003
1995
2009


In [8]:
ppt_daily = xr.open_mfdataset(str(DAILY_TOTAL_OUT / '*_daily_precipitation.nc'), 
                              chunks={'latitude': 121, 
                                  'longitude': 144})

In [9]:
ppt_5day = ppt_daily.tp.rolling(time=5).sum()
ppt_5day.name = 'precipitation'
ppt_5day = ppt_5day.to_dataset()

In [10]:
years, datasets = zip(*ppt_5day.groupby('time.year'))
paths = [PPT_5DAY_OUT / f'{y}_ppt_5day.nc' for y in years]
xr.save_mfdataset(datasets, paths)